In [42]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [43]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


False

In [44]:
# print(stopwords.words('english'))

# Data Preprocessing

In [45]:
# load the fake news dataset into a pandas dataframe -- train.csv
# news_data = pd.read_csv("/Users/balogunishaq/Desktop/Complete Machine Learning Course - Siddhardhan/datasets/diabetes.csv")
news_data = pd.read_csv("/Users/balogunishaq/Desktop/Complete Machine Learning Course - Siddhardhan/datasets/Life Expectancy Data.csv")

In [46]:
news_data.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


# 0 --> Yes
# 1 --> No

In [47]:
news_data.shape

(2938, 22)

In [48]:
# Checking for missing values
news_data.isnull().sum()

Country                              0
Year                                 0
Status                               0
Life expectancy                     10
Adult Mortality                     10
infant deaths                        0
Alcohol                            194
percentage expenditure               0
Hepatitis B                        553
Measles                              0
 BMI                                34
under-five deaths                    0
Polio                               19
Total expenditure                  226
Diphtheria                          19
 HIV/AIDS                            0
GDP                                448
Population                         652
 thinness  1-19 years               34
 thinness 5-9 years                 34
Income composition of resources    167
Schooling                          163
dtype: int64

In [49]:
# Replace missing values with a null string
news_data = news_data.fillna('')

In [50]:
# Merging country and status 
news_data['Content'] = news_data['Country']+ ' '+news_data['Status']

In [51]:
news_data.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Content
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,8.16,65.0,0.1,584.25921,33736494.0,17.2,17.3,0.479,10.1,Afghanistan Developing
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,Afghanistan Developing
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.47,9.9,Afghanistan Developing
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,8.52,67.0,0.1,669.959,3696958.0,17.9,18.0,0.463,9.8,Afghanistan Developing
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,Afghanistan Developing


In [52]:
# Separating Features and Target
X = news_data.drop(columns='Alcohol', axis=1)
Y = news_data['Alcohol']

In [53]:
print(X)

          Country  Year      Status Life expectancy  Adult Mortality  \
0     Afghanistan  2015  Developing             65.0           263.0   
1     Afghanistan  2014  Developing             59.9           271.0   
2     Afghanistan  2013  Developing             59.9           268.0   
3     Afghanistan  2012  Developing             59.5           272.0   
4     Afghanistan  2011  Developing             59.2           275.0   
...           ...   ...         ...              ...             ...   
2933     Zimbabwe  2004  Developing             44.3           723.0   
2934     Zimbabwe  2003  Developing             44.5           715.0   
2935     Zimbabwe  2002  Developing             44.8            73.0   
2936     Zimbabwe  2001  Developing             45.3           686.0   
2937     Zimbabwe  2000  Developing             46.0           665.0   

      infant deaths  percentage expenditure Hepatitis B  Measles   BMI   ...  \
0                62               71.279624        65.0

In [54]:
print(Y)

0       0.01
1       0.01
2       0.01
3       0.01
4       0.01
        ... 
2933    4.36
2934    4.06
2935    4.43
2936    1.72
2937    1.68
Name: Alcohol, Length: 2938, dtype: object


# Stemming

# This is the process of reducing words to its Root Word

In [55]:
port_stem = PorterStemmer()

In [56]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    return stemmed_content

In [57]:
# news_data['Content'] = news_data['Content'].apply(stemming)

In [58]:
# print(news_data['Content'])

In [59]:
# X = news_data['content'].values
# Y = news_data['label'].values

In [60]:
# print(X)

In [61]:
# print(Y)

In [62]:
Y.shape

(2938,)

In [65]:
# converting the textual data into feature vectors
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)